This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [186]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'z9GxWfpN1t5q5n6aKLb1'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [187]:
# First, import the relevant modules
import requests
from datetime import date

Setting up functions to make it easier to query metadata on stocks and get data for a given period. At present we are not doing any error handling.

In [188]:
def getmetadata(database, stock):
    # Setup url to get metadata for given stock
    payload = {'api_key': API_KEY}
    meta_url = 'https://www.quandl.com/api/v3/datasets/' + database + '/' + stock + '_X/metadata.json'
    
    # Call Quandl API and return response as a dict if request is successful
    response = requests.get(meta_url, params = payload)
    
    # Check HTTP response status
    if response.ok == False:
        return null
    else:
        # format response text as JSON 
        return response.json()

In [189]:
def getdataset (database,stock,start_date,end_date):
    # setup url to get dataset for given stock 
    data_url = 'https://www.quandl.com/api/v3/datasets/' + database + '/' + stock + '_X/data.json'    

    # setup query parameters 
    payload = {'start_date':start_date, 
               'end_date': end_date, 
               'api_key': API_KEY}
    
    # Call Quandl API and return response as a dict if request is successful
    response = requests.get(data_url, params = payload)
    
    # Check HTTP response status
    if response.ok == False:
        return null
    else:
        # format response text as JSON 
        return response.json()    

Now that the functions are setup, let's inspect what data is available for AFX and the data structure

In [190]:
# Get metadata to inspect the information available on a stock 
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
metadata = getmetadata('FSE','AFX')
print(metadata)

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-12-01T14:48:09.907Z', 'newest_available_date': '2020-12-01', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'database_id': 6129}}


In [191]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
stock_prices = getdataset('FSE','AFX','2017-01-01','2017-01-03')
print(stock_prices)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-01-01', 'end_date': '2017-01-03', 'frequency': 'daily', 'data': [['2017-01-03', 35.9, 35.93, 35.34, 35.48, None, 70618.0, 2515473.0, None, None, None], ['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]], 'collapse': None, 'order': None}}


Stock prices are provided as a list of lists within a dict. We need to extract the values for the 'data' key in this dict

In [192]:
print((stock_prices['dataset_data']['data']))
print(stock_prices['dataset_data']['data'][0])

[['2017-01-03', 35.9, 35.93, 35.34, 35.48, None, 70618.0, 2515473.0, None, None, None], ['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]]
['2017-01-03', 35.9, 35.93, 35.34, 35.48, None, 70618.0, 2515473.0, None, None, None]


Let's define a function to extract the given prices or volumes from the JSON object returned by the API call
Encapsulating this into a function makes it easier to keep data structure specific code in one place

In [193]:
def get_data(dataset, col_name):
    
    # Get column index for given column name
    col_idx = dataset['dataset_data']['column_names'].index(col_name)
        
    # Extract all rows for the given column from the 'data' column in JSON 
    # We use a nested list comprehension
    return [ day[col_idx] for day in [days for days in dataset['dataset_data']['data']] ]    

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

**Tasks 1,2 are implemented within functions get_dataset() and get_data()**

In [194]:
stock_data = getdataset('FSE','AFX','2017-01-01','2017-12-31')

In [195]:
print(' Traded volume (sample): ',get_data(stock_data,'Traded Volume')[0:5])
print(' High prices (sample): ',get_data(stock_data,'High')[0:5])

 Traded volume (sample):  [34640.0, 40660.0, 57452.0, 71165.0, 120649.0]
 High prices (sample):  [51.94, 51.82, 51.89, 51.5, 51.52]


**3. Calculate what the highest and lowest opening prices were for the stock in this period.**

In [196]:
opening_prices = get_data(stock_data,'Open')
print('Highest opening price was', max(filter(None, opening_prices)),'and lowest',min(filter(None, opening_prices)) )

Highest opening price was 53.11 and lowest 34.0


**4. What was the largest change in any one day (based on High and Low price)?**

In [233]:
high_prices = get_data(stock_data,'High')
low_prices = get_data(stock_data,'Low')
daily_price_change = [ high - low for high,low in zip(high_prices,low_prices) ]  
print('The largest change in any one day (based on High and Low price) was', max(daily_price_change))

The largest change in any one day (based on High and Low price) was 2.8100000000000023


**5. What was the largest change between any two days (based on Closing Price)?**

In [235]:
closing_prices = get_data(stock_data,'Close')
close_price_delta = [nxt_day - prev_day for nxt_day, prev_day in zip(closing_prices[1:len(closing_prices)], closing_prices[0:len(high_prices)-1])]
print('The largest change between any two days (based on Closing Price) was', max(close_price_delta))

The largest change between any two days (based on Closing Price) was 2.559999999999995


**6. What was the average daily trading volume during this year?**

In [199]:
trade_vols = get_data(stock_data,'Traded Volume')
avg_trade_vol = sum(trade_vols)/len(trade_vols)
print('The average daily trading volume during this year was',round(avg_trade_vol))

The average daily trading volume during this year was 89124


**7. What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)**

In [232]:
# create shallow copy of list before sorting
sorted_vols = list(filter(None,trade_vols.copy()))
sorted_vols.sort()
# calculate median
list_length = len(sorted_vols)//2
if len(sorted_vols) % 2 == 1:
    print('The median trading volume during this year was',sorted_vols[list_length])
else:
    print('The median trading volume during this year was',(sorted_vols[list_length] + sorted_vols[list_length-1])/2)

The median trading volume during this year was 76286.0
